In [ ]:
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cf
import numpy as np

In [ ]:
# import fields and grid

%store -r fields
[uWind,vWind,wWind,temp,geop,div,vort,geop_height] = fields

%store -r grid
[lon,lat,pressure_levels] = grid

In [ ]:
# for calculating the gradient of a given field, first the distance between
# grid points is converted to metres. the distance in meridional direction is
# constant (deg_to_m) but varies in zonal direction depending on the latitude (dist)

deg_to_m = 111120

dist = np.array(deg_to_m * np.cos(np.deg2rad(lat)))
dist = np.swapaxes(np.tile(dist,[len(lon),1]),0,1)

def grad(field):
    '''this function calculates the gradient of field'''
    grad = np.zeros((2,*temp.isel(time=0).shape))
    
    for j in range(1,len(field.latitude)-1):
        grad[1,:,j,:] = - 1 / (2 * deg_to_m) * (field.isel(time=1)[:,j+1,:] - field.isel(time=1)[:,j-1,:])
    grad[1,:,0,:] = - 1 / deg_to_m * (field.isel(time=1)[:,1,:] - field.isel(time=1)[:,0,:])
    grad[1,:,-1,:] = - 1 / deg_to_m * (field.isel(time=1)[:,-1,:] - field.isel(time=1)[:,-2,:])
    
    for i in range(1,len(field.longitude)-1):
        grad[0,:,:,i] = 1 / (2 * dist[:,i]) * (field.isel(time=1)[:,:,i+1] - field.isel(time=1)[:,:,i-1])
    grad[0,:,:,0] = 1 / dist[:,0] * (field.isel(time=1)[:,:,1] - field.isel(time=1)[:,:,0])
    grad[0,:,:,-1] = 1 / dist[:,-1] * (field.isel(time=1)[:,:,-1] - field.isel(time=1)[:,:,-2])
    
    return grad

In [ ]:
# calculate the coriolis parameter f and set a critical latitude lat_c
# at which the geostrophic wind relation breaks down bc of the 1/f term

lat_c = 10

omega = 7.2921e-5
f = np.zeros((len(lat),len(lon)))
f = 2 * omega * np.sin(np.deg2rad(lat))
f = np.swapaxes(np.tile(f, (len(lon),1)),0,1)
f[90-lat_c:90+lat_c] = np.nan
recip_f = 1 / f

The geostrophic velocity is calculated according to
\begin{align}
\vec{v}_g = \dfrac{1}{f} \vec{k} \times \nabla_p \Phi
\end{align}

In [ ]:
uWindG = - recip_f * grad(geop)[1]
vWindG =   recip_f * grad(geop)[0]

In [ ]:
# store geostrophic wind so it can be imported into other notebooks

%store uWindG
%store vWindG

In [ ]:
def geos_wind(field,N=90,S=90,W=0,E=360,pressure_level=-2,spacing=5,scale=500):
    '''this function plots field with an areal extend of [N,S,W,E] at pressure_level
       with the wind and geostrophic wind displayed as arrows.
       W is given in degrees east and has to be smaller than E,
       also no negative values are allowed.
       spacing is the space inbetween arrows in degrees.
       scale gives the length of the arrows in the image. try out
       different values, smaller values give longer arrows.
       the geopotential height is displayed as contour lines (in m).
    '''
    N = 90-N
    S = 90+S

    fig, ax = plt.subplots(figsize=(15,8), subplot_kw={'projection': ccrs.PlateCarree()})
    im = ax.contourf(lon[W:E], lat[N:S], field[1,pressure_level,N:S,W:E],
                    cmap='viridis', levels=20)
    
    im2 = ax.contour(lon[W:E], lat[N:S], geop_height[1,pressure_level,N:S,W:E])
    ax.clabel(im2, im2.levels, inline=True,colors='k')

    Q = ax.quiver(lon[W:E][::spacing], lat[N:S][::spacing],
                   uWind[1,pressure_level,N:S,W:E][::spacing,::spacing],
                   vWind[1,pressure_level,N:S,W:E][::spacing,::spacing],
                   color = "r",scale=scale)
    Qk = ax.quiverkey(Q, 0.5,-0.1,np.nanmax(uWind[1,pressure_level,N:S,W:E][::spacing,::spacing]),
                      label="{:.0f}".format(np.nanmax(uWind[1,pressure_level,N:S,W:E][::spacing,::spacing])) + " m/s wind velocity",
                      labelpos = 'E')

    Q2 = ax.quiver(lon[W:E][::spacing], lat[N:S][::spacing],
                  uWindG[pressure_level,N:S,W:E][::spacing,::spacing],
                  vWindG[pressure_level,N:S,W:E][::spacing,::spacing],
                  color="b",scale=scale)
    Qk2 = ax.quiverkey(Q2,0.5,-0.15,np.nanmax(uWind[1,pressure_level,N:S,W:E][::spacing,::spacing]),
                      label="{:.0f}".format(np.nanmax(uWind[1,pressure_level,N:S,W:E][::spacing,::spacing])) + " m/s geostrophic wind velocity",
                      labelpos = 'E')

    ax.add_feature(cf.COASTLINE)
    ax.add_feature(cf.BORDERS)
    ax.set_xticks([0],[0])
    ax.set_yticks([0],[0])

    col = fig.colorbar(im, orientation='horizontal', fraction=0.039*len(lon)/len(lat), label=f"{field.long_name} [{field.units}]")
    ax.set_title(f"{field.long_name} at pressure level {pressure_levels[pressure_level]} hPa", fontsize=20)
    fig.tight_layout()

### Use the function to compare the geostrophic wind with the actual wind in different regions and at different altitudes

also try out different values for spacing and scale

In [ ]:
pressure_levels # in hPa

In [ ]:
geos_wind(temp,pressure_level=-3,spacing=10,scale=400)

In [ ]:
geos_wind(temp,N=60,S=-40,W=0,E=20,spacing=1,scale=500,pressure_level=-4)